### Extraction des mots

In [2]:
import re
import json

# Chemin du fichier texte
txt_file_path = 'glyph.txt'

# Lire le fichier texte
with open(txt_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Liste pour stocker les traductions
translations = []

# Regex améliorée pour capturer toutes les lignes, y compris les mots multiples
for line in lines:
    # Vérifie si la ligne contient un mot anglais et une traduction valyrienne
    match = re.match(r"#(\w+)#.*?>\s*([\w,\s-]+)", line)
    if match:
        english_word = match.group(1).strip()  # Mot anglais (début de ligne)
        valyrian_words = [word.strip() for word in match.group(2).split(",")]  # Mot(s) valyriens (fin de ligne)

        # Ajouter chaque mot valyrien avec le mot anglais
        for valyrian_word in valyrian_words:
            if valyrian_word:  # S'assurer qu'il n'y a pas d'entrée vide
                translations.append({"english": english_word, "valyrian": valyrian_word})

# Enregistrer dans un fichier JSON
output_path = 'valyrian_to_english.json'
with open(output_path, 'w', encoding='utf-8') as json_file:
    json.dump(translations, json_file, indent=4, ensure_ascii=False)

print(f"Traductions enregistrées dans le fichier : {output_path}")

Traductions enregistrées dans le fichier : valyrian_to_english.json


### Création du corpus

Test du modèle de classement de genre

In [3]:
import spacy

# Charger un modèle pré-entraîné (anglais par exemple)
nlp = spacy.load("en_core_web_sm")

# Liste de mots à analyser
words = ["run", "beautiful", "quickly", "tree", "I"]

for word in words:
    doc = nlp(word)
    for token in doc:
        print(f"Word: {token.text}, POS: {token.pos_}")


Word: run, POS: VERB
Word: beautiful, POS: ADJ
Word: quickly, POS: ADV
Word: tree, POS: NOUN
Word: I, POS: PRON


Generation du corpus

In [5]:
import spacy
import pandas as pd
import json
import random

# Charger SpaCy
nlp = spacy.load("en_core_web_sm")

# Charger un fichier JSON de vocabulaire
def load_vocab(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Ajouter la colonne 'genre' (part-of-speech) avec SpaCy
def enrich_vocab_with_pos(vocab):
    enriched_data = []
    for entry in vocab:
        english_word = entry['english']
        valyrian_word = entry['valyrian']

        # Analyser le mot anglais pour en déduire la catégorie grammaticale
        doc = nlp(english_word)
        pos = doc[0].pos_  # Catégorie grammaticale du premier token

        enriched_data.append({
            "english": english_word,
            "valyrian": valyrian_word,
            "pos": pos
        })

    # Retourner un DataFrame enrichi
    return pd.DataFrame(enriched_data)

# Générer un corpus parallèle structuré
def generate_corpus(dataframe, num_sentences=10):
    corpus = []
    for _ in range(num_sentences):
        try:
            # Sélectionner des mots pour chaque rôle grammatical
            subject_row = dataframe[dataframe['pos'] == 'NOUN'].sample(1)
            subject = subject_row['valyrian'].values[0]
            subject_translation = subject_row['english'].values[0]

            verb_row = dataframe[dataframe['pos'] == 'VERB'].sample(1)
            verb = verb_row['valyrian'].values[0]
            verb_translation = verb_row['english'].values[0]

            obj_row = dataframe[dataframe['pos'] == 'NOUN'].sample(1)
            obj = obj_row['valyrian'].values[0]
            obj_translation = obj_row['english'].values[0]

            adjective_row = dataframe[dataframe['pos'] == 'ADJ'].sample(1) if 'ADJ' in dataframe['pos'].values else None
            adjective = adjective_row['valyrian'].values[0] if adjective_row is not None else ""
            adjective_translation = adjective_row['english'].values[0] if adjective_row is not None else ""

            # Construire une phrase selon la structure SVO ou SOV avec adjectifs et prépositions
            fictional_sentence = f"{subject} {verb} {adjective} {obj} ".strip()
            translation_sentence = f"{subject_translation} {verb_translation} {adjective_translation} {obj_translation}".strip()

            # Ajouter au corpus
            corpus.append({"input": fictional_sentence, "output": translation_sentence})
        except Exception as e:
            print(f"Erreur lors de la génération d'une phrase : {e}")
    return corpus


# Sauvegarder un DataFrame ou un corpus en JSON
def save_to_json(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# Fonction principale
def main():

    # Charger le vocabulaire
    vocab = load_vocab("valyrian_to_english.json")

    # Enrichir le vocabulaire avec les catégories grammaticales
    df = enrich_vocab_with_pos(vocab)
    print("DataFrame enrichi :")
    print(df)

    # Générer un corpus parallèle aléatoire
    corpus = generate_corpus(df, num_sentences=10000)
    print("Corpus généré :")
    print(corpus)

    # Sauvegarder le corpus en JSON
    save_to_json(corpus, "corpus.json")



In [6]:
if __name__ == "__main__":
    main()

DataFrame enrichi :
      english  valyrian   pos
0     ability   kostion  NOUN
1        able   kostion   ADJ
2       above     tolie   ADP
3    absolute   tatagon  VERB
4    absolute      teta  VERB
..        ...       ...   ...
759      word      udir  NOUN
760     world       vȳs  NOUN
761     worth   kostion   ADJ
762     write  bardugon  VERB
763    writer  bardugon  NOUN

[764 rows x 3 columns]
Corpus généré :
[{'input': 'qilōny indigon jelmio kostion', 'output': 'whip intend northern champion'}, {'input': 'pragron suez bōsa limagon', 'output': 'casket jammed tall tear'}, {'input': 'perzys iōragon klios pelar', 'output': 'fire receive quick trouble'}, {'input': 'grevy suez kostion somby', 'output': 'wheel wedged worth fort'}, {'input': 'gaba havor quba muña', 'output': 'monkey sustenance poor mom'}, {'input': 'gūron pryjagon morghon drakarys', 'output': 'information broken dead dragon'}, {'input': 'endien kēli naena vȳs', 'output': 'sunset kitten many planet'}, {'input': 'limagon